### This is the code to access Brazilian Central Bank API to obtain macroeconomic data
#### References: 
- Filho, L. H. B. (2022, April 11). Coletando dados do Banco Central com Python. Análise Macro. https://analisemacro.com.br/economia/indicadores/coletando-dados-do-banco-central-com-python/.
Contribution: The basic approach to access Brazilian Central Bank API.


#### Data sources
- Brazil. (2024). SGS - Sistema Gerenciador de Séries Temporais. https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries

#### Libraries
- Freitas, W. (2023). Python-bcb (0.2.0) [Python]. https://pypi.org/project/python-bcb/ - Freitas, W. (2021). Documentação python-bcb. Documentação python-bcb. https://wilsonfreitas.github.io/python-bcb/
- Package Pandas (2.2). (2024). [Python]. https://pandas.pydata.org/

In [1]:
# Import libraries

import pandas as pd
from bcb import sgs

import useful_functions as uf

In [2]:
# Define the start date
start_date = '2000-01-31'

# Set the series to be downloaded
data_series = {'RP_month':4639, # Get the primary result series - In R$ millions. Includes data from Central Government in Brazil: Federal Government + INSS + BACEN
               'IPCA_month_433': 433, # Monthly IPCA - Brazil's official inflation index
               'IPCA_month_13522_anual': 13522, # 12-months IPCA - Brazil's official inflation index
               'selic':432, # Selic (basic interest rate in Brazil) rate % per year
               'dbgg_%_pre':4537, # Gross Debt (% of GDP) - General Government (Federal, State and Municipal). Pre stands for a method used until 2007
               'dbgg_%_pos':13762, # Gross Debt (% of GDP) - General Government (Federal, State and Municipal). Pos stands for a method used since 2008
               'dbgg_R$_pre':4502, # Gross Debt (R$ millions) - General Government (Federal, State and Municipal). Pre stands for a method used until 2007               
               'dbgg_R$_pos':13761, # Gross Debt (R$ millions) - General Government (Federal, State and Municipal). Pos stands for a method used since 2008
               'dlgg_%':4536, # Net Debt (% of GDP) - General Government (Federal, State and Municipal)
               'dlgg_R$':4501,# Net Debt (R$ millions) - General Government (Federal, State and Municipal)
               'dlfg_%':4503, # Net Debt (% of GDP) - Federal Government + Central Bank
               'dlfg_R$':4468, # Net Debt (R$ millions) - Federal Government + Central Bank
               'dlspc_R$':4513, # Net Debt (%) - Federal Government + Central Bank - Consolidated Public Sector
               'bal_payments_US$_M':22707,# Balance on goods - Balance of Payments - monthly - net
               'GDP_R$_12_months':4382,# 	GDP accumulated in the last 12 months - current prices (R$ million)
               'IR_US$':3546,# 	International reserves (US$ million) - Monthly
               } 
# Download the data
df_raw = sgs.get(data_series, start = start_date)

# Adjust the index to be a datetime object
df_raw.index = pd.to_datetime(df_raw.index)

In [3]:
# Create a copy of the raw data filtering only the last available day of each month
df_macro = df_raw.resample('ME').last().copy()

# Rename the columns
df_macro = df_macro.rename(columns={
    'RP_month': 'ECO_fiscal_result_month',
    'IPCA_month_433': 'ECO_inflation_month',
    'IPCA_month_13522_anual': 'ECO_inflation_12months',
    'selic': 'ECO_interest_rate',
    'dbgg_%_pre': 'ECO_gross_debt_gdp_pre',
    'dbgg_%_pos': 'ECO_gross_debt_gdp_pos',
    'dbgg_R$_pre': 'ECO_gross_debt_R$_pre',
    'dbgg_R$_pos': 'ECO_gross_debt_R$_pos',
    'dlgg_%': 'ECO_net_debt_gdp_%',
    'dlgg_R$': 'ECO_net_debt_R$',
    'dlfg_%': 'ECO_net_debt_gdp_%_federal_govt',
    'dlfg_R$': 'ECO_net_debt_R$_federal_govt',
    'dlspc_R$': 'ECO_net_debt_R$_dlspc',
    'bal_payments_US$_M': 'ECO_balance_payments_US$_M',
    'GDP_R$_12_months': 'ECO_GDP_R$_12_months',
    'IR_US$': 'ECO_inter_reserves_US$'
})

# Invert values from RP so that negative means deficit and positive means surplus
df_macro['ECO_fiscal_result_month'] = df_macro['ECO_fiscal_result_month'] * -1

# Compute the sum of last 12 months for each row
df_macro['ECO_fiscal_result_12months'] = df_macro['ECO_fiscal_result_month'].rolling(window=12).sum()

# Desired column order
column_order = ['ECO_fiscal_result_month', 'ECO_fiscal_result_12months', 
                'ECO_inflation_month', 'ECO_inflation_12months', 
                'ECO_interest_rate', 
                'ECO_gross_debt_gdp_pre', 'ECO_gross_debt_gdp_pos', 'ECO_gross_debt_R$_pre', 'ECO_gross_debt_R$_pos', 
                'ECO_net_debt_gdp_%', 'ECO_net_debt_R$', 'ECO_net_debt_gdp_%_federal_govt', 'ECO_net_debt_R$_federal_govt', 'ECO_net_debt_R$_dlspc',
                'ECO_balance_payments_US$_M', 'ECO_GDP_R$_12_months','ECO_inter_reserves_US$']

# Reindex the DataFrame with the desired column order
df_macro = df_macro.reindex(columns=column_order)


In [5]:
# Convert the dataset to monthly frequency getting the last valid value of each month
df_macro_monthly = uf.convert_df_to_monthly(df_macro,'%Y-%m-%d')
# Export the dataset
df_macro_monthly.to_csv('../data/df_macroeco.csv', index=True)
